In [6]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import LeakyReLU, Reshape, BatchNormalization, Conv2DTranspose, Conv2D,UpSampling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import random
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm
import tempfile
from PIL import Image
import keras
import numpy as np
from math import ceil
import os
os.environ['CUDA_CACHE_PATH'] = 'D:/cuda_cache'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Tắt tất cả các thông báo log


In [7]:
def upsample_block(
    x,
    filters,
    activation,
    kernel_size=(5,5),
    strides=(1,1),
    up_size=(2,2),
    padding='same',
    use_bn=True,
    use_bias=False,
    use_dropout=False,
    drop_value=0.3
):
    x = UpSampling2D(up_size)(x)
    x = Conv2D(
        filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias
    )(x)

    if use_bn:
        x = BatchNormalization()(x)
    if activation:
        x = activation(x)
    if use_dropout:
        x = Dropout(drop_value)(x)

    return x

In [8]:
def build_generator_model(
    z_dim = 128, 
    n_filter = 64
):
    z_input = Input(shape=(z_dim,))

    # Dense 2*2*n_filter*8
    x = Dense(2*2*n_filter*8, use_bias=True)(z_input)

    # 2*2*512
    x = Reshape((2,2,n_filter*8))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    # 4*4*256
    x = upsample_block(
        x,
        filters=4*n_filter,
        activation=LeakyReLU(0.2),
        kernel_size=(5,5),
        strides=(1,1),
        up_size=(2,2),
        padding='same',
        use_bn=True,
        use_bias=True,
        use_dropout=False,
        drop_value=0.3
    )
    

    # 8*8*128
    x = upsample_block(
        x,
        filters=2*n_filter,
        activation=LeakyReLU(0.2),
        kernel_size=(5,5),
        strides=(1,1),
        up_size=(2,2),
        padding='same',
        use_bn=True,
        use_bias=True,
        use_dropout=False,
        drop_value=0.3
    )

    # 16*16*64
    x = upsample_block(
        x,
        filters=n_filter,
        activation=LeakyReLU(0.2),
        kernel_size=(5,5),
        strides=(1,1),
        up_size=(2,2),
        padding='same',
        use_bn=True,
        use_bias=True,
        use_dropout=False,
        drop_value=0.3
    )
    # 32*32*3
    x = upsample_block(
        x,
        filters=3,
        activation=Activation('tanh'),
        kernel_size=(5,5),
        strides=(1,1),
        up_size=(2,2),
        padding='same',
        use_bn=True,
        use_bias=True,
        use_dropout=False,
        drop_value=0.3
    )
    
    g_model = Model(z_input, x, name="generator")

    return g_model

In [9]:
def con_block(
    x,
    filters,
    activation,
    kernel_size=(3,3),
    strides=(1,1),
    padding="same",
    use_bias=True,
    use_bn=False,
    use_dropout=False,
    drop_value=0.5
):
    x = Conv2D(
        filters, kernel_size, strides=strides, padding=padding, use_bias=use_bias
    )(x)

    if use_bn:
        x = BatchNormalization()(x)

    if activation:
        x = activation(x)

    if use_dropout:
        x = Dropout(drop_value)(x)

    return x

In [10]:
def build_discriminator_model( 
    input_shape=(32,32,3), 
    n_filter=64
):
    input = Input(shape=input_shape)
    
    # 16*16*64
    x = con_block(
        input,
        filters=n_filter,
        activation=LeakyReLU(0.2),
        kernel_size=(5,5),
        strides=(2,2),
        padding="same",
        use_bias=True,
        use_bn=True,
        use_dropout=False,
        drop_value=0.5
    )

    # 8*8*128
    x = con_block(
        x,
        filters=2*n_filter,
        activation=LeakyReLU(0.2),
        kernel_size=(5,5),
        strides=(2,2),
        padding="same",
        use_bias=True,
        use_bn=True,
        use_dropout=False,
        drop_value=0.5
    )
    
    # 4*4*256
    x = con_block(
        x,
        filters=4*n_filter,
        activation=LeakyReLU(0.2),
        kernel_size=(5,5),
        strides=(2,2),
        padding="same",
        use_bias=True,
        use_bn=True,
        use_dropout=False,
        drop_value=0.5
    )
    
    # 2*2*512
    x = con_block(
        x,
        filters=8*n_filter,
        activation=LeakyReLU(0.2),
        kernel_size=(5,5),
        strides=(2,2),
        padding="same",
        use_bias=True,
        use_bn=True,
        use_dropout=False,
        drop_value=0.5
    )
    
    x = Flatten()(x)
    x = Dense(1)(x)
    x = Activation('sigmoid')(x)
    
    d_model = Model(input, x, name="discriminator")
    return d_model

In [23]:
class DCGANs(Model):
    def __init__(
        self, 
        generator,
        discriminator,
        latent_dim=128,
    ):
        super().__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.latent_dim = latent_dim
        self.seed_generator = tf.random.Generator.from_seed(1337)

    def compile(self, optimizer_discriminator, optimizer_generator, loss_d, loss_g):
        super().compile()
        self.g_optimizer = optimizer_generator
        self.d_optimizer = optimizer_discriminator
        self.loss_d = loss_d
        self.loss_g = loss_g

    def train_step(self, batch_image):
        if isinstance(batch_image, tuple):
            batch_image = batch_image[0]

        batch_size = tf.shape(batch_image)[0]
        # Tạo vector latent
        random_latent_vectors = self.seed_generator.normal(shape=(batch_size, self.latent_dim))
        
        # Tạo ảnh giả 
        generated_images = self.generator(random_latent_vectors)
        
        # Gán nhãn giả là 0 và nhãn thật là 1
        y_fake = tf.zeros((batch_size, 1)) + 0.05 * tf.random.uniform(tf.shape(tf.zeros((batch_size, 1))))
        y_real = tf.ones((batch_size, 1)) + 0.05 * tf.random.uniform(tf.shape(tf.ones((batch_size, 1))))

        # Huấn luyện Discriminator với ảnh thật
        with tf.GradientTape() as tape:
            y_pred_real = self.discriminator(batch_image, training=True)
            d_loss_r = self.loss_d(y_real, y_pred_real)
        
        d_gradient_real = tape.gradient(d_loss_r, self.discriminator.trainable_variables)
        self.d_optimizer.apply_gradients(zip(d_gradient_real, self.discriminator.trainable_variables))

        # Huấn luyện Discriminator với ảnh giả
        with tf.GradientTape() as tape:
            y_pred_fake = self.discriminator(generated_images, training=True)
            d_loss_f = self.loss_d(y_fake, y_pred_fake)

        d_gradient_fake = tape.gradient(d_loss_f, self.discriminator.trainable_variables)
        self.d_optimizer.apply_gradients(zip(d_gradient_fake, self.discriminator.trainable_variables))

        # Huấn luyện Generator
        with tf.GradientTape() as tape:
            y_pred = self.discriminator(generated_images, training=True)
            g_loss = self.loss_g(y_real, y_pred)

        g_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        self.g_optimizer.apply_gradients(zip(g_gradient, self.generator.trainable_variables))

        # Tính tổng loss của Discriminator
        d_loss = (d_loss_r + d_loss_f) / 2
        return {"d_loss": d_loss, "g_loss": g_loss}


In [24]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim
        self.seed_generator =  keras.random.SeedGenerator(42)

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(
            shape=(self.num_img, self.latent_dim), seed=self.seed_generator
        )
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = keras.utils.array_to_img(generated_images[i])
            img.save("generated_img_%03d_%d.png" % (epoch, i))

In [25]:
def get_data(data_name):
    (X_train, _), (_, _) = cifar10.load_data()
    # Chuyển hình ảnh từ (0 -> 255) về ( -1 -> 1) 
    X_train = X_train.astype(np.float32)
    X_train = 2*(X_train/255)-1
    return X_train

In [29]:
def binary_crossentropy(y_real, y_pred):
    # Tránh log(0) bằng cách thêm epsilon
    epsilon = 1e-7
    y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
    
    # Công thức binary crossentropy
    return -(y_real * tf.math.log(y_pred) + (1 - y_real) * tf.math.log(1 - y_pred))

In [30]:
lr_d = 2e-4
lr_g = 2e-4
beta_g = 0.5
beta_d = 0.5
epochs = 20
latent_dim = 128
BATCH_SIZE = 128

optimizer_generator = Adam(learning_rate=lr_g, beta_1=beta_g)
optimizer_discriminator = Adam(learning_rate=lr_d, beta_1=beta_d)

d_model = build_discriminator_model()
g_model = build_generator_model()

cbk = GANMonitor(num_img=3, latent_dim=latent_dim)

# Get the dcGans model
dcgans = DCGANs(
    discriminator=d_model,
    generator=g_model,
    latent_dim=latent_dim,
)

# Compile the wgan model
dcgans.compile(
    optimizer_discriminator=optimizer_discriminator,
    optimizer_generator=optimizer_generator,
    loss_d=binary_crossentropy,
    loss_g=binary_crossentropy,
)

X_train = get_data(cifar10)

In [31]:
# Start training
dcgans.fit(X_train, batch_size=BATCH_SIZE, epochs=epochs, callbacks=[cbk])

Epoch 1/20


ValueError: No gradients provided for any variable.